In [112]:
import pandas as pd
import zeep

In [113]:
df = pd.read_csv('symptoms.csv', sep=',',encoding='latin-1')

In [114]:
df.reset_index(drop=True)

,Unnamed: 0,[PathogenMitKrankheit].[Meldeweg Web71].[Meldeweg ID Web71].[MEMBER_CAPTION],[PathogenMitKrankheit].[Meldeweg Web71].[Krankheit ID Web71].[MEMBER_CAPTION],[Symptome].[ID].[ID].[MEMBER_CAPTION],[ReportingDate].[YearWeek].[YearWeek].[MEMBER_CAPTION],[Measures].[FallCount]
0,1,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2015-KW13,1
1,2,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2015-KW33,1
2,3,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2015-KW44,1
3,4,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2015-KW53,1
4,5,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW05,1
5,6,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW06,5
6,7,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW07,3
7,8,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW08,4
8,9,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW10,7
9,10,Gemäß IfSG,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW11,5


In [115]:
df = df.drop(columns=['Unnamed: 0','[PathogenMitKrankheit].[Meldeweg Web71].[Meldeweg ID Web71].[MEMBER_CAPTION]'])

In [116]:
df = df.rename(columns={'[PathogenMitKrankheit].[Meldeweg Web71].[Krankheit ID Web71].[MEMBER_CAPTION]': 'Krankheit', '[Symptome].[ID].[ID].[MEMBER_CAPTION]': 'Symptom', '[ReportingDate].[YearWeek].[YearWeek].[MEMBER_CAPTION]': 'Kalenderwoche', '[Measures].[FallCount]': 'Anzahl'}) 
df

,Krankheit,Symptom,Kalenderwoche,Anzahl
0,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2015-KW13,1
1,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2015-KW33,1
2,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2015-KW44,1
3,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2015-KW53,1
4,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW05,1
5,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW06,5
6,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW07,3
7,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW08,4
8,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW10,7
9,Adenovirus-K(eratok)onjunktivitis,Karunkelschwellung,2016-KW11,5


In [117]:
df_selection = df[df.isin({'Krankheit': ['Norovirus-Gastroenteritis','Influenza', 'Windpocken']})['Krankheit']]
df_selection

,Krankheit,Symptom,Kalenderwoche,Anzahl
54046,Influenza,akuter Krankheitsbeginn,2003-KW41,1
54047,Influenza,akuter Krankheitsbeginn,2004-KW14,1
54048,Influenza,akuter Krankheitsbeginn,2004-KW42,1
54049,Influenza,akuter Krankheitsbeginn,2004-KW43,2
54050,Influenza,akuter Krankheitsbeginn,2004-KW44,1
54051,Influenza,akuter Krankheitsbeginn,2004-KW45,2
54052,Influenza,akuter Krankheitsbeginn,2004-KW47,1
54053,Influenza,akuter Krankheitsbeginn,2004-KW48,1
54054,Influenza,akuter Krankheitsbeginn,2004-KW49,2
54055,Influenza,akuter Krankheitsbeginn,2004-KW50,2


In [118]:
#df_selection_symptoms = df_selection.groupby(['Krankheit','Symptom'])['Anzahl'].sum().reset_index()
#df_selection_symptoms

In [119]:
df_selection = df_selection.drop(columns='Krankheit')
df_selection

,Symptom,Kalenderwoche,Anzahl
54046,akuter Krankheitsbeginn,2003-KW41,1
54047,akuter Krankheitsbeginn,2004-KW14,1
54048,akuter Krankheitsbeginn,2004-KW42,1
54049,akuter Krankheitsbeginn,2004-KW43,2
54050,akuter Krankheitsbeginn,2004-KW44,1
54051,akuter Krankheitsbeginn,2004-KW45,2
54052,akuter Krankheitsbeginn,2004-KW47,1
54053,akuter Krankheitsbeginn,2004-KW48,1
54054,akuter Krankheitsbeginn,2004-KW49,2
54055,akuter Krankheitsbeginn,2004-KW50,2


In [120]:
df_selection_pivot = df_selection.groupby(['Symptom','Kalenderwoche'])['Anzahl'].sum().reset_index()
df_selection_pivot.sort_values('Kalenderwoche')

,Symptom,Kalenderwoche,Anzahl
3118,Husten,2001-KW01,2
6105,andere Symptome,2001-KW01,1
2055,Fieber,2001-KW01,3
3904,"Muskel-, Glieder-, Rücken- oder Kopfschmerzen",2001-KW01,1
253,Ausschlag an Haut oder Schleimhaut mit gleichz...,2001-KW01,10
3905,"Muskel-, Glieder-, Rücken- oder Kopfschmerzen",2001-KW02,5
3119,Husten,2001-KW02,3
254,Ausschlag an Haut oder Schleimhaut mit gleichz...,2001-KW02,15
2056,Fieber,2001-KW02,6
3120,Husten,2001-KW03,7


In [121]:
#get a list of all relevant symptoms
df_selection_all_symptoms = df_selection.groupby(['Symptom'])['Anzahl'].sum().reset_index()['Symptom'].tolist()
df_selection_all_symptoms 

['Ausschlag an Haut oder Schleimhaut mit Flecken, Bläschen oder Pusteln (außer Herpes zoster)',
 'Ausschlag an Haut oder Schleimhaut mit gleichzeitig vorhandenen Papeln, Bläschen bzw. Pusteln und Schorf (sog. Sternenhimmel)',
 'Ausschlag, einseitig auf Hautsegment beschränkt, bläschenförmig',
 'Brennen, Juckreiz',
 'Durchfall und/oder Erbrechen',
 'Durchfall, nicht näher bezeichnet',
 'Fetales (kongenitales) Varizellensyndrom',
 'Fieber',
 'Gliederschmerzen',
 'Husten',
 'Muskel-, Glieder-, Rücken- oder Kopfschmerzen',
 'Pneumonie (Lungenentzündung)',
 'Schmerzen im betroffenen Bereich (Zosterneuralgie)',
 'Schmerzen, einseitig auf ein Hautsegment lokalisiert, ohne Ausschlag',
 'akuter Krankheitsbeginn',
 'akutes schweres Atemnotsyndrom (ARDS)',
 'andere Symptome',
 'beatmungspflichtige Atemwegserkrankung']

In [132]:
#for each combination of symptom and week:
#if the combination is not in the dataframe, then insert a row with this combination and 'Anzahl' = 0
new_rows = []
for week in df['Kalenderwoche'].drop_duplicates().sort_values():
    for symptom in df_selection_all_symptoms:
        sample_week = df_selection_pivot[df_selection_pivot['Kalenderwoche'] == week]
        if not sample_week.isin({'Symptom': [symptom]})['Symptom'].any():
            new_row = [symptom, week, 0]
            new_rows.append(new_row)
            
            
new_values = pd.DataFrame(new_rows, columns=['Symptom','Kalenderwoche','Anzahl'])
                        

In [133]:
df_selection_pivot= df_selection_pivot.append(new_values)

In [140]:
df_weekly_symptoms = df_selection_pivot.sort_values('Kalenderwoche').reset_index(drop=True)